## Librerias.

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime, timedelta
import re

## Carga de datos

In [2]:
data_origin = pd.read_excel('Puntualidad Bogotá.xlsx')
datos = DataFrame(data_origin.copy())
datos.head(10)

,Fecha / Hora,Asunto
0,2022-06-01T11:07:56+00:00,<7569|20220601> Shein - Control Horas
1,2022-06-01T11:25:34+00:00,<7865|20220601> Messer Consolidado Correos
2,2022-06-01T11:26:32+00:00,<6886|20220601> Messer Ocupación Agente
3,2022-06-01T11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio
4,2022-06-01T11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile
5,2022-06-01T11:37:53+00:00,<7776|20220601> BBVA - BBVA EMPRESAS Recuperac...
6,2022-06-01T11:51:01+00:00,<6885|20220601> Messer 5/15
7,2022-06-01T12:00:26+00:00,<7745|20220601> Pichincha IVR
8,2022-06-01T12:00:30+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...
9,2022-06-01T12:17:21+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...


## Tipos de datos.

In [3]:
datos.dtypes

Fecha / Hora    object
Asunto          object
dtype: object

## Dimenciones del dataframe.

In [4]:
datos.shape

(5383, 2)

## Datos faltantes.

In [5]:
datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
datos_faltantes

Fecha / Hora    0
Asunto          0
dtype: int64

## Porcentajes de datos faltantes por columna.

In [6]:
for column in datos.columns:
    porcentaje_faltantes = datos[column].isnull().mean()
    print(column, '=', round(porcentaje_faltantes*100,2))

Fecha / Hora = 0.0
Asunto = 0.0


## Convertir la columna tipo object a datetime64 en una nueva columna.

In [7]:
datos['Fecha / Hora'] = pd.to_datetime(datos['Fecha / Hora'])

datos.dtypes

Fecha / Hora    datetime64[ns, UTC]
Asunto                       object
dtype: object

In [8]:
datos.head(10)

,Fecha / Hora,Asunto
0,2022-06-01 11:07:56+00:00,<7569|20220601> Shein - Control Horas
1,2022-06-01 11:25:34+00:00,<7865|20220601> Messer Consolidado Correos
2,2022-06-01 11:26:32+00:00,<6886|20220601> Messer Ocupación Agente
3,2022-06-01 11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio
4,2022-06-01 11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile
5,2022-06-01 11:37:53+00:00,<7776|20220601> BBVA - BBVA EMPRESAS Recuperac...
6,2022-06-01 11:51:01+00:00,<6885|20220601> Messer 5/15
7,2022-06-01 12:00:26+00:00,<7745|20220601> Pichincha IVR
8,2022-06-01 12:00:30+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...
9,2022-06-01 12:17:21+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...


## Identificar los ID.

In [9]:
datos['ID'] = datos['Asunto'].str.partition('|')[0].str.extract('(\d{4})')
datos[495:505]

,Fecha / Hora,Asunto,ID
495,2022-06-02 18:46:42+00:00,<6621|20220602> Ceet Penetración Combos Junio ...,6621
496,2022-06-02 19:00:09+00:00,<7815|20220602> ETB CD Reporte de Tiempos,7815
497,2022-06-02 19:03:38+00:00,<8028|20220602> Banmedica - Tablero de Control...,8028
498,2022-06-02 19:04:01+00:00,<7835|20220602> Sena - Redes Sociales,7835
499,2022-06-02 19:07:37+00:00,<7748|20220602> Liberty Informe Coomeva,7748
500,2022-06-02 19:12:39+00:00,<7459|20220602> BBVA - Reporte Irene Seguros,7459
501,2022-06-02 19:13:14+00:00,Informe Bo <6532|20220602>,6532
502,2022-06-02 19:28:54+00:00,<|20220602> Sena - Reporte de Satisfacción de ...,NaN
503,2022-06-02 19:30:40+00:00,<7940|20220602> Ceet Informe Ventas caídas - r...,7940
504,2022-06-02 19:31:13+00:00,RE: <7459|20220602> BBVA - Reporte Irene Seguros,7459


## Identificando los datos faltantes por ID.

In [10]:
datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
datos_faltantes

Fecha / Hora     0
Asunto           0
ID              37
dtype: int64

## Eliminar fila de ID faltante.

In [11]:

# datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
# datos_faltantes
datos = datos[~(datos['ID'].isnull())]

## Porcentajes de ID faltantes

In [12]:
for column in datos.columns:
    porcentaje_faltantes = datos[column].isnull().mean()
    print(column, '=', round(porcentaje_faltantes*100,2))

Fecha / Hora = 0.0
Asunto = 0.0
ID = 0.0


## Extraer comentario.

In [13]:
# extraer_comentario = lambda x: "".join(re.findall("[a-z A-Z]",str(x).strip()))

extraer_comentario = lambda x: re.sub("\<.*?\>","",x)
datos['Comentario'] = datos['Asunto'].apply(extraer_comentario).str.strip()

## Eliminamos la columna 'Asunto'.

In [14]:
# datos = datos.drop(datos.columns[[1]], axis=1)
datos = datos.drop(columns=['Asunto'])
datos.to_csv('DatosLimpios.csv')
datos

,Fecha / Hora,ID,Comentario
0,2022-06-01 11:07:56+00:00,7569,Shein - Control Horas
1,2022-06-01 11:25:34+00:00,7865,Messer Consolidado Correos
2,2022-06-01 11:26:32+00:00,6886,Messer Ocupación Agente
3,2022-06-01 11:30:51+00:00,6887,Messer Niveles De Servicio
4,2022-06-01 11:33:21+00:00,6887,Messer Niveles De Servicio Chile
...,...,...,...
5378,2022-06-30 20:00:10+00:00,7408,BBVA - Reporte Tipificación Outbound
5379,2022-06-30 20:00:46+00:00,8061,Cruz Verde Tablero De Control Unificado Junio ...
5380,2022-06-30 20:01:10+00:00,6874,Cruz Verde Actualización Kpi's Junio 20220629
5381,2022-06-30 20:01:21+00:00,7689,Medicarte Tablero De Control Junio 20220629


In [15]:
#datos = datos.set_index('ID')
datos

,Fecha / Hora,ID,Comentario
0,2022-06-01 11:07:56+00:00,7569,Shein - Control Horas
1,2022-06-01 11:25:34+00:00,7865,Messer Consolidado Correos
2,2022-06-01 11:26:32+00:00,6886,Messer Ocupación Agente
3,2022-06-01 11:30:51+00:00,6887,Messer Niveles De Servicio
4,2022-06-01 11:33:21+00:00,6887,Messer Niveles De Servicio Chile
...,...,...,...
5378,2022-06-30 20:00:10+00:00,7408,BBVA - Reporte Tipificación Outbound
5379,2022-06-30 20:00:46+00:00,8061,Cruz Verde Tablero De Control Unificado Junio ...
5380,2022-06-30 20:01:10+00:00,6874,Cruz Verde Actualización Kpi's Junio 20220629
5381,2022-06-30 20:01:21+00:00,7689,Medicarte Tablero De Control Junio 20220629


## Eliminamos los ID duplicados.

In [16]:
datos.drop_duplicates()
datos[495:505]

,Fecha / Hora,ID,Comentario
497,2022-06-02 19:03:38+00:00,8028,Banmedica - Tablero de Control Banmédica Mayo
498,2022-06-02 19:04:01+00:00,7835,Sena - Redes Sociales
499,2022-06-02 19:07:37+00:00,7748,Liberty Informe Coomeva
500,2022-06-02 19:12:39+00:00,7459,BBVA - Reporte Irene Seguros
501,2022-06-02 19:13:14+00:00,6532,Informe Bo
503,2022-06-02 19:30:40+00:00,7940,Ceet Informe Ventas caídas - recuperadas
504,2022-06-02 19:31:13+00:00,7459,RE: BBVA - Reporte Irene Seguros
505,2022-06-02 19:32:48+00:00,7836,Sena - Transcript Chat
506,2022-06-02 19:34:26+00:00,7562,Ceet - Ausentismos Suscripciones
507,2022-06-02 19:42:14+00:00,7714,Publicacion Simple Codigos Detallados
